In [0]:
# from keras.applications.resnet50 import ResNet50
# from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16, preprocess_input
# from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
# from keras.applications.resnet50 import decode_predictions#, preprocess_input

from keras import optimizers
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Flatten, Input, Dropout
from keras.layers import Conv2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D, MaxPooling2D
# from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

# from tensorflow.python.keras.applications import ResNet50
# from tensorflow.python.keras.applications import Xception
# from tensorflow.python.keras.models import Sequential
# from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
# from tensorflow.python.keras.applications.resnet50 import preprocess_input
# from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

# import cv2
import numpy as np
# import pandas as pd
import os
from PIL import Image
from matplotlib import pyplot as plt

# import tensorflow as tf

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# !cp "drive/My Drive/Colab Notebooks/classified.zip" classified.zip
# !cp "drive/My Drive/Colab Notebooks/sample_submission.csv" sample_submission.csv

In [0]:
# !mv test classified/classified/test/test

In [0]:
# !unzip classified.zip -d classified/



Data Generators

In [85]:
target_size = (224, 224)
batch_size = 32
num_classes = 196


data_generator_with_aug = ImageDataGenerator(
        rotation_range=10, 
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.3,
        ##testing##
        brightness_range=(0.8, 1.2),
        channel_shift_range=0.1,

         
        horizontal_flip=True)

train_generator = data_generator_with_aug.flow_from_directory(
        'classified/classified/train',
        target_size=target_size,
        batch_size=24,
        class_mode='categorical')

data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = data_generator_no_aug.flow_from_directory(
        'classified/classified/validation',
        target_size=target_size,
        class_mode='categorical')

test_generator = data_generator_no_aug.flow_from_directory(
        'classified/classified/test',
        shuffle=False,
        
        target_size=target_size)

Found 11834 images belonging to 196 classes.
Found 1000 images belonging to 196 classes.
Found 3305 images belonging to 1 classes.


VGG16 as backbone, 1 conv and 2 dense layers + Batchnorm and Dropout

In [0]:
model = Sequential()
model.add(VGG16(include_top=False, input_shape=(224, 224, 3), weights='imagenet'))
model.add(BatchNormalization())

model.add(Conv2D(input_shape=(7, 7, 2048), 
                 data_format='channels_last', kernel_size = (3,3), strides=(2,2), 
                 filters=1024, activation='relu', padding='valid'))
          
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.layers[0].trainable = False
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
batch_normalization_7 (Batch (None, 7, 7, 512)         2048      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 1024)        4719616   
_________________________________________________________________
batch_normalization_8 (Batch (None, 3, 3, 1024)        4096      
_________________________________________________________________
dropout_4 (Dropout)          (None, 3, 3, 1024)        0         
_________________________________________________________________
global_average_pooling2d_4 ( (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)             

Training.
Save and lower learning rate every 5 epochs

In [0]:
optimizer = optimizers.Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [39]:
h = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=5,
        validation_data=validation_generator,
        validation_steps=len(validation_generator))

Epoch 1/2
494/494 [==============================] - 190s 384ms/step - loss: 2.1221 - acc: 0.4621 - val_loss: 3.0562 - val_acc: 0.3130
Epoch 2/2
494/494 [==============================] - 187s 378ms/step - loss: 1.7501 - acc: 0.5369 - val_loss: 2.7283 - val_acc: 0.3570


In [0]:
model.save("drive/My Drive/Colab Notebooks/vgg16_conv_2fc_5ep.h5")

In [0]:
optimizer = optimizers.Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [66]:
h = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=10,
        validation_data=validation_generator,
        validation_steps=len(validation_generator))

Epoch 1/5
494/494 [==============================] - 205s 416ms/step - loss: 0.5608 - acc: 0.8328 - val_loss: 1.1648 - val_acc: 0.6880
Epoch 2/5
494/494 [==============================] - 202s 409ms/step - loss: 0.4935 - acc: 0.8514 - val_loss: 1.2587 - val_acc: 0.6410
Epoch 3/5
494/494 [==============================] - 203s 410ms/step - loss: 0.4850 - acc: 0.8559 - val_loss: 1.4193 - val_acc: 0.6240
Epoch 4/5
494/494 [==============================] - 203s 410ms/step - loss: 0.4003 - acc: 0.8780 - val_loss: 1.5009 - val_acc: 0.6240
Epoch 5/5
494/494 [==============================] - 203s 411ms/step - loss: 0.4834 - acc: 0.8578 - val_loss: 1.0626 - val_acc: 0.7010


In [0]:
model.save("drive/My Drive/Colab Notebooks/vgg16_conv_2fc_15ep.h5")

In [0]:
optimizer = optimizers.Adam(lr=0.00005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [70]:
h = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=5,
        validation_data=validation_generator,
        validation_steps=len(validation_generator))

Epoch 1/5
494/494 [==============================] - 208s 421ms/step - loss: 0.2935 - acc: 0.9124 - val_loss: 0.9496 - val_acc: 0.7470
Epoch 2/5
494/494 [==============================] - 203s 412ms/step - loss: 0.2238 - acc: 0.9344 - val_loss: 0.7609 - val_acc: 0.7970
Epoch 3/5
494/494 [==============================] - 204s 412ms/step - loss: 0.2527 - acc: 0.9261 - val_loss: 0.9967 - val_acc: 0.7330
Epoch 4/5
494/494 [==============================] - 205s 414ms/step - loss: 0.2150 - acc: 0.9360 - val_loss: 0.7146 - val_acc: 0.8070
Epoch 5/5
494/494 [==============================] - 205s 415ms/step - loss: 0.1353 - acc: 0.9636 - val_loss: 0.6363 - val_acc: 0.8230


In [0]:
model.save("drive/My Drive/Colab Notebooks/vgg16_conv_2fc_20ep.h5")

In [0]:
optimizer = optimizers.Adam(lr=0.000025)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
h = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=5,
        validation_data=validation_generator,
        validation_steps=len(validation_generator))

In [0]:
model.save("drive/My Drive/Colab Notebooks/vgg16_conv_2fc_25ep.h5")

In [0]:
optimizer = optimizers.Adam(lr=0.00001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
h = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=5,
        validation_data=validation_generator,
        validation_steps=len(validation_generator))

In [0]:
model.save("drive/My Drive/Colab Notebooks/vgg16_conv_2fc_30ep.h5")

In [0]:
optimizer = optimizers.Adam(lr=0.000005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
h = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=5,
        validation_data=validation_generator,
        validation_steps=len(validation_generator))

In [0]:
model.save("drive/My Drive/Colab Notebooks/vgg16_conv_2fc_35ep.h5")

Final training on validation

In [116]:
h = model.fit_generator(
        validation_generator,
        steps_per_epoch=len(validation_generator),
        epochs=5)

Epoch 1/5
32/32 [==============================] - 9s 287ms/step - loss: 0.6194 - acc: 0.8301
Epoch 2/5
32/32 [==============================] - 8s 236ms/step - loss: 0.3322 - acc: 0.9120
Epoch 3/5
32/32 [==============================] - 8s 235ms/step - loss: 0.2243 - acc: 0.9306
Epoch 4/5
32/32 [==============================] - 8s 236ms/step - loss: 0.1489 - acc: 0.9658
Epoch 5/5
32/32 [==============================] - 8s 235ms/step - loss: 0.1305 - acc: 0.9687


In [117]:
h = model.fit_generator(
        validation_generator,
        steps_per_epoch=len(validation_generator),
        epochs=5)

Epoch 1/5
32/32 [==============================] - 8s 237ms/step - loss: 0.1076 - acc: 0.9757
Epoch 2/5
32/32 [==============================] - 7s 234ms/step - loss: 0.0885 - acc: 0.9835
Epoch 3/5
32/32 [==============================] - 8s 235ms/step - loss: 0.0655 - acc: 0.9902
Epoch 4/5
32/32 [==============================] - 8s 235ms/step - loss: 0.0556 - acc: 0.9913
Epoch 5/5
32/32 [==============================] - 8s 235ms/step - loss: 0.0686 - acc: 0.9855


In [0]:
optimizer = optimizers.Adam(lr=0.000001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [120]:
h = model.fit_generator(
        validation_generator,
        steps_per_epoch=len(validation_generator),
        epochs=5)

Epoch 1/5
32/32 [==============================] - 13s 400ms/step - loss: 0.0569 - acc: 0.9923
Epoch 2/5
32/32 [==============================] - 8s 236ms/step - loss: 0.0439 - acc: 0.9971
Epoch 3/5
32/32 [==============================] - 8s 236ms/step - loss: 0.0475 - acc: 0.9990
Epoch 4/5
32/32 [==============================] - 8s 236ms/step - loss: 0.0422 - acc: 0.9961
Epoch 5/5
32/32 [==============================] - 8s 236ms/step - loss: 0.0377 - acc: 0.9971


In [0]:
optimizer = optimizers.Adam(lr=0.0000005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [124]:
h = model.fit_generator(
        validation_generator,
        steps_per_epoch=len(validation_generator),
        epochs=5)

Epoch 1/5
32/32 [==============================] - 13s 413ms/step - loss: 0.0449 - acc: 0.9952
Epoch 2/5
32/32 [==============================] - 8s 235ms/step - loss: 0.0357 - acc: 0.9980
Epoch 3/5
32/32 [==============================] - 8s 235ms/step - loss: 0.0426 - acc: 0.9961
Epoch 4/5
32/32 [==============================] - 8s 235ms/step - loss: 0.0421 - acc: 0.9952
Epoch 5/5
32/32 [==============================] - 8s 236ms/step - loss: 0.0360 - acc: 0.9990


In [0]:
model.save("drive/My Drive/Colab Notebooks/vgg16_conv_2fc_final.h5")

In [134]:
X, y = next(validation_generator)
model.evaluate(X, y)

32/32 [==============================] - 0s 3ms/step


[0.026479989290237427, 1.0]

probably overfitting

In [0]:
optimizer = optimizers.Adam(lr=0.0000001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [136]:
h = model.fit_generator(
        validation_generator,
        steps_per_epoch=len(validation_generator),
        epochs=5)

Epoch 1/5
32/32 [==============================] - 8s 239ms/step - loss: 0.0335 - acc: 0.9980
Epoch 2/5
32/32 [==============================] - 8s 236ms/step - loss: 0.0435 - acc: 0.9990
Epoch 3/5
32/32 [==============================] - 8s 237ms/step - loss: 0.0276 - acc: 0.9990
Epoch 4/5
32/32 [==============================] - 8s 235ms/step - loss: 0.0347 - acc: 0.9961
Epoch 5/5
32/32 [==============================] - 8s 236ms/step - loss: 0.0369 - acc: 0.9952


In [138]:
h = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=3)

Epoch 1/3
494/494 [==============================] - 206s 418ms/step - loss: 0.0415 - acc: 0.9904
Epoch 2/3
494/494 [==============================] - 204s 412ms/step - loss: 0.0488 - acc: 0.9890
Epoch 3/3
494/494 [==============================] - 203s 410ms/step - loss: 0.0398 - acc: 0.9896


Generating predictions

In [0]:
preds = model.predict_generator(
        test_generator)

In [127]:
preds.shape

(3305, 196)

In [0]:
pred_labels = [x.argmax() for x in preds]

In [135]:
np.array(pred_labels)

array([ 70,   0, 124, ...,  73, 162,  95])

In [0]:
s = pd.read_csv('sample_submission.csv')
s['label'] = pred_labels
s.to_csv('submission.csv', index=False, index_label = False)

In [1]:


images = []
for i in range(100,120):
  label = s.label[i]
  im_file = s.image[i]
  first_image_in_dir = sorted(os.listdir(os.path.join('classified/classified/train', str(label).zfill(4))))[0]
  images.append(Image.open(os.path.join('classified/classified/test', 'test', im_file)))
  images.append(Image.open(os.path.join('classified/classified/train', str(label).zfill(4), first_image_in_dir)))
  
fig=plt.figure(figsize=(15, 40))
columns = 2
rows = 10
for i in range(1, columns*rows+1):
    img = images[i-1]
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

NameError: ignored